In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding = "ISO-8859-1")
data.head()

In [ ]:
data.tail()

I think our data is v1 and v2

In [ ]:
sms=data["v2"]
y=data["v1"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sms, y, test_size=0.33, random_state=2001)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
X_train=tokenizer.texts_to_sequences(x_train)
X_test=tokenizer.texts_to_sequences(x_test)

In [ ]:
maxlen=-11
for sms in X_train:
    candidatemax=len(sms)
    if maxlen<=candidatemax:
        maxlen=candidatemax
print(maxlen)        

In [ ]:
maxlentest=-11
for sms in X_test:
    candidatemax=len(sms)
    if maxlentest<=candidatemax:
        maxlentest=candidatemax
print(maxlentest)       

I select 200 as sequence length

In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen=200
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 
print(vocab_size)

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_train=le.fit_transform(y_train)
Y_test=le.transform(y_test)

In [ ]:
model.fit(X_train, Y_train,
                    epochs=15,
                    verbose=True,
                    validation_data=(X_test, Y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, Y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
ypred=model.predict(X_test)
print(confusion_matrix(y_true=Y_test,y_pred=ypred>0.5))

In [ ]:
le.classes_

In [ ]:
le.inverse_transform([0])

okay 0 is ham 1 is spam

What is false positive rate

In [ ]:
tn, fn, fp, tp = confusion_matrix(y_true=Y_test,y_pred=ypred>0.5).ravel()

In [ ]:
fpr=fp/(fp+tn)
print("false positive rate: ",fpr)

## False Positive Rate is very very good!

### Neural Network works very very good in this dataset.